In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, date_format, year, month, dayofweek, minute, second,
    to_date, date_add
)
from pyspark.sql.types import TimestampType
import pandas as pd
import time

# Initialize Spark session
spark = SparkSession.builder.appName("DatetimeFunctionsStandalone").getOrCreate()

In [ ]:
# Generate sample timestamps (simulated microseconds since epoch)
now = time.time()
sample_data = [
    {"user_id": 1, "timestamp": int((now - 10000) * 1e6)},
    {"user_id": 2, "timestamp": int((now - 5000) * 1e6)},
    {"user_id": 3, "timestamp": int((now - 1000) * 1e6)},
    {"user_id": 4, "timestamp": int(now * 1e6)},
]

In [ ]:
# Convert to DataFrame
pdf = pd.DataFrame(sample_data)
df = spark.createDataFrame(pdf)
df.show(truncate=False)

In [ ]:
# Cast to proper timestamp type
timestamp_df = df.withColumn("timestamp", (col("timestamp") / 1e6).cast(TimestampType()))
timestamp_df.show(truncate=False)

In [ ]:
# Format date and time
formatted_df = (timestamp_df
    .withColumn("date_string", date_format("timestamp", "MMMM dd, yyyy"))
    .withColumn("time_string", date_format("timestamp", "HH:mm:ss.SSSSSS"))
)
formatted_df.show(truncate=False)

In [ ]:
# Extract parts from timestamp
datetime_df = (timestamp_df
    .withColumn("year", year(col("timestamp")))
    .withColumn("month", month(col("timestamp")))
    .withColumn("dayofweek", dayofweek(col("timestamp")))
    .withColumn("minute", minute(col("timestamp")))
    .withColumn("second", second(col("timestamp")))
)
datetime_df.show(truncate=False)

In [ ]:
# Convert to Date
date_df = timestamp_df.withColumn("date", to_date(col("timestamp")))
date_df.show(truncate=False)

In [ ]:
# Date manipulation (add days)
plus_2_df = timestamp_df.withColumn("plus_two_days", date_add(col("timestamp"), 2))
plus_2_df.show(truncate=False)

In [ ]:
# Stop session
spark.stop()